# Acquisition program used to capture data during experiment

The program use the U3 board to pilote the data acquisition
The first part of the program set the Input and the output of the card

DAC 0 is used to pilote the valve with a voltage varying between 0V and 5V. An operational amplifier is used
the double the tension to vary between 0V and 10V because the valve controler expect this variation for the voltage

The pump motor is a brushless DC, the pump is provided with an electronic control to pilote the velocity of the pump with 10 positions available.

FIO4 Measure the voltage of the Pump
FIO5 Measure the intensity consumed by the pump.

A second operational amplifier is used to amplify the voltage measured to to the shunt resitor. This voltage is the image of the intensity consumed by the motor. The intensity is measured at the entry of the electronic control piloting the motor.

We configure the counter 1 that will be used the pulse delivered by the flow meter. The flow meter provides 97 pulses / litter. We use a window period of 10 seconds to count the pulses. The counter is reset for each flow measure.

After the setup, the valve is piloted to test the order are received correctly by the valve controler.
The flow measure is done to check the value sent by the flow meter.

The datasheet are available on the document directory associated to this capstone project.
You will found:
- Brushless DC principle
- Valve model
- Valve controler
- Labjack U3 datasheet
- Flow Meter data sheet
- Operational amplifier data sheet


In [135]:
import LabJackPython
import u3
d=u3.U3()
#d.debug=True
#d.configIO(FIOAnalog=1)
#d.writeRegister(5000,3)
#d.getAIN(0,32)
#d.configU3()
#LabJackPython.listAll(3)
#print d.configU3()
#help(u3.WaitLong)

# Switch OFF the LED
d.getFeedback(u3.LED(State=False))
# Switch ON the LED
d.getFeedback(u3.LED(State=True))

# DAC voltage output variation between 0 and 5V
# Control DAC 0
# DAC 0 is used to pilote the valve simulatating the filter clogging
# Plug The valve controller
# Gray = variable command 0 -> 10V,  black = GND , red = 24v
# Controller needs 75s to go to 0 to 100%
# Control vary from 0x00 to 0xff

d.getFeedback(u3.DAC8(Dac=0,Value=0xff))

# Control DAC 1
# d.getFeedback(u3.DAC8(Dac=1,Value=0xFF))

# Channel 0-7 => AIN0-AIN7 (FIO0 - FIO7)
# Channel 8-15 => AIN8-AIN15 (EIO0 - EIO7)

# SET FIO4 and FIO5 as ananolg input
# FIO4 Measure the voltage of the Pump
# FIO5 Measure the intensity consumed by the pump

# Pump is BLDC model with 3 phases

d.configIO(FIOAnalog = 0x30)

# Configure counter 1
# Set Offset to 6
# The flow metter provides 97 impulses / liter
# Plug The flow metter wire
# Green = Pulse Signal, Braun = +5V , White = GND
# Set a pull-up resistor on pulse signal (Output is an open collector)
d.configIO(EnableCounter1 = True, NumberOfTimersEnabled =1, TimerCounterPinOffset = 6)

# Print the config
print d.configIO()

#Read FIO4
# NegativeChannel=31 means single ended
# The entry voltage vary from 0 to 2,44v
print d.getFeedback(u3.AIN(PositiveChannel=4,NegativeChannel=31,LongSettling=False,QuickSample=False))
#Read FIO5
# NegativeChannel=31 means single ended
# The entry voltage vary from 0 to 2,44v
print d.getFeedback(u3.AIN(PositiveChannel=5,NegativeChannel=31,LongSettling=False,QuickSample=False))



{'NumberOfTimersEnabled': 1, 'TimerCounterPinOffset': 6, 'DAC1Enable': 0, 'FIOAnalog': 63, 'EIOAnalog': 0, 'TimerCounterConfig': 105, 'EnableCounter1': True, 'EnableCounter0': False}
[10144]
[11488]


In [140]:
#test valve command
d.getFeedback(u3.DAC8(Dac=0,Value=0xff))

[None]

In [137]:
#test Read counter 1 with reset
print d.getFeedback(u3.Counter(counter = 1, Reset = True))

[35]


In [138]:
#test measure intensity
current=0
for cnt in range(0,20):
    print cnt
    cur = d.getFeedback(u3.AIN(PositiveChannel=5,NegativeChannel=31,LongSettling=False,QuickSample=False))[0]
    print cur
    current= current = current + cur
current = current /20
print current

0
27504
1
26368
2
14720
3
10256
4
15552
5
24160
6
15584
7
8656
8
25520
9
23456
10
15456
11
7808
12
25200
13
23776
14
15712
15
7856
16
15856
17
9472
18
11408
19
24816
17456


# Program piloting the acquisition

For each cycle the velocity of the pump and the load at the output of the pump.

The velocity is fixed manualy on the pump electronic control vary from 1 to 10
The output load of the pump simulate the distribution circuit of the water and is controled by a manual valve 
- 100%
- 75%
- 50%
- 25%

For each cycle the valve start from a position completly open (command = 255 => 10v) to a position completly closed (0 => 0v) and the value decrease by 5
After be completly closed the valve open from 0 to 255 by step of 5.

For each value of the entry valve load the acquisition system capture :

- The cycle number
- The current time
- The water flow (given by the flow metter)
- The load entry sent to the valve (simulate the filter clogging)
- The load at the output (fixed for a cycle)
- The current consumed by the pump
- The pump velocity (fixed for a cycle)

The result of each measure is stored in a pandas dataframe saved in csv format.

In [139]:
import pandas as pd
import datetime
import time

def measureCycle(cycle, d, df, loadEntryStart, loadEntryEnd, step, pumpVelocity, outputLoad):
    # For a cycle, pumpVelocity and outputLoad remain constant
    loadEntry = loadEntryStart
    # Put the loadEntry at start position
    d.getFeedback(u3.DAC8(Dac=0,Value=loadEntry))
    
    if loadEntryStart == 255 :
        # Wait loadentry
        print 'Put loadEntry at : ',loadEntryStart
        valveEntryOk = raw_input('Entry Valve is ready (Y/N) ')
 
    while (loadEntry <= loadEntryEnd and step > 0) or (loadEntry >= loadEntryEnd and step < 0):
        # Measure the Intensity (take an average for 20 Measures)
        pumpCurrent= 0
        for cnt in range(0,20):
            cur = d.getFeedback(u3.AIN(PositiveChannel=5,NegativeChannel=31,LongSettling=False,QuickSample=False))[0]
            pumpCurrent = current + cur
        pumpCurrent = pumpCurrent /20
        # Read flow to put the counter to zero
        d.getFeedback(u3.Counter(counter = 1, Reset = True))
        # Wait 10 seconds (count the pulse provided by the flow metter for a period of 10s)
        time.sleep(10)
        flow= d.getFeedback(u3.Counter(counter = 1, Reset = True))[0]
        # Take the current time
        curTime= datetime.datetime.now()
    
        # Write in dataframe record (time, flow, load entry, load output, intensity, pumpvelocity)
        df.loc[len(df)]=[cycle, curTime, flow, loadEntry, outputLoad, pumpCurrent, pumpVelocity]
        
        print 'Cycle:        ',cycle
        print 'Time:         ',curTime
        print 'Flow:         ',flow
        print 'Load Entry:   ',loadEntry
        print 'Load Output:  ',outputLoad
        print 'Pump current: ',pumpCurrent
        print 'Pump velocity:',pumpVelocity
        print '--------------------'
    
        # Adjust the load entry
        loadEntry = loadEntry + step
        # Put the new loadentry
        d.getFeedback(u3.DAC8(Dac=0,Value=loadEntry))
        # Wait 3 s
        time.sleep(3)

# Initiate and create the dataframe
columns = ['Cycle', 'DateTime', 'Flow', 'EntryLoad', 'OutputLoad', 'PumpCurrent', 'PumpVelocity']
df = pd.DataFrame(columns=columns)

exitFlag=False
cycle=4;

while not exitFlag:
    cycle= cycle+1
    print 'Cycle Nr: ',cycle
    # Ask the values for the Pump Velocity and Load Output
    pumpVelocity = input('Pump velocity (1-10)') 
    outputLoad =   input('Output load (100,75,50,25) ')

    #Cycle closing the valve
    measureCycle(cycle, d, df, 255, 0, -5, pumpVelocity, outputLoad)
    #Cycle opening the valve
    measureCycle(cycle, d, df, 0, 255, 2, pumpVelocity, outputLoad)
   
    selection = raw_input('New measure cycle? (Y,N) ')
    if selection == 'N':
        exitFlag=True

print 'DataFrame is saving in a CSV file'        
#Save the dataframe as a CSV file
df.to_csv('PumpMeasures.csv')
print 'End of the measure process'

Cycle Nr:  5
Pump velocity (1-10)6
Output load (100,75,50,25) 100
Put loadEntry at :  255
Entry Valve is ready (Y/N) Y
Cycle:         5
Time:          2017-04-23 17:53:49.554000
Flow:          62
Load Entry:    255
Load Output:   100
Pump current:  1464
Pump velocity: 6
--------------------
Cycle:         5
Time:          2017-04-23 17:54:02.598000
Flow:          63
Load Entry:    250
Load Output:   100
Pump current:  1400
Pump velocity: 6
--------------------
Cycle:         5
Time:          2017-04-23 17:54:15.641000
Flow:          66
Load Entry:    245
Load Output:   100
Pump current:  1944
Pump velocity: 6
--------------------
Cycle:         5
Time:          2017-04-23 17:54:28.687000
Flow:          66
Load Entry:    240
Load Output:   100
Pump current:  1631
Pump velocity: 6
--------------------
Cycle:         5
Time:          2017-04-23 17:54:41.718000
Flow:          68
Load Entry:    235
Load Output:   100
Pump current:  2320
Pump velocity: 6
--------------------
Cycle:         5